# Splinepy
## The Library
### Splinepy
> Python N-dimensional Bezier, Rational Bezier, BSpline and NURBS library with C++ backend 
It provides all low-level functionality for splines, including
 - import / export for various formats, such as Irit, gismo, XML, iges, JSON and MFEM
 - Order Elevation
 - Evaluation
 - Knot Insertion (NURBS, BSplines)
 - Derivatives
 - Compositions of Bezier patches
 - Multiplication and Addition of Bezier-Extracted elements
 - Multipatch fields and boundary identification for pre-processing
 
Splinepy depends on a modified version of `SplineLib`, `Napf`, `Uff` and `Bezman`. Here, `SplineLib` is used for NURBS and BSplines, `Bezman` handles all things Bezier. Further `scipy` is regarded as an optional dependency and is somtimes used if available for sparse matrices.

### Other libraries
If you are interested in exploring the C++ backend, we refer to:
- [Splinelib](https://github.com/tataratat/splinelib)
- [BezMan](https://github.com/tataratat/bezman)
- [Napf](https://github.com/tataratat/napf)
- [uff](https://github.com/tataratat/uff)

In particular, we would like to interest you in checking out our splinepy-wrapper `gustaf`, that extends splinepy for plotting capabilities and facilitates the construction of spline-geometries using hands-on functions. You can find the newest version [here](https://github.com/tataratat/gustaf) of install it (together with splinepy) using:
```
pip install gustaf[all]
```

## Getting Started
### Basics


In [ ]:
import splinepy as sp
import numpy as np

All Spline-Types are constructed using (positional or) the following keyword arguments:
- `degrees` (list)
- `control_points` (list[list] oder `numpy` type)
- `knot_vectors` (list[list])
- `weights`(list of `numpy`-types)

The keywords are used where applicable, i.e.:

| | NURBS | BSpline | Bezier | Rational Bezier |
| -: | :-: | :-: | :-: | :-: |
| `degrees` | X | X | X | X |
| `control_points` | X | X | X | X |
| `knot_vectors` | X | X | - | - |
| `weights` | X | - | - | X |

Let's create some splines:

In [ ]:
# Bezier types
bezier_line = sp.Bezier(degrees=[1], control_points=[[0.0, 0.0], [2, 1]])
bezier_surface = sp.Bezier(
    degrees=[1, 1], control_points=[[0.0, 1.0], [2, 3], [0, 2], [2, 4]]
)

# BSpline
bspline_cube = sp.BSpline(
    degrees=[1, 1, 1],
    control_points=[
        [0, 0, 0],
        [1, 0, 0],
        [0, 1, 0],
        [1, 1, 0],
        [0, 0, 1],
        [1, 0, 1],
        [0, 1, 1],
        [1, 1, 1],
    ],
    knot_vectors=[[0, 0, 1, 1], [0, 0, 1, 1], [0, 0, 1, 1]],
)

# NURBs
nurbs_line = sp.NURBS(
    degrees=[2],
    knot_vectors=[[0, 0, 0, 0.5, 1, 1, 1]],
    control_points=[[0.0, 1.0], [2, 3], [0, 2], [2, 4]],
    weights=[1, 0.5, 0.8, 1],
)

### Manipulation
Basic spline refinement strategies are avaible, to modify the parametrization. These include knot insertion and degree elevation, as well as knot removal and degree reduction, where applicable.

#### Degree elevation

In [ ]:
# Elevate the degree along one specific parametric dimension
bezier_line.elevate_degrees(0)

# Or multiple times at once
bspline_cube.elevate_degrees([0, 1, 2, 0])

#### Knot insertion


In [ ]:
bspline_cube.insert_knots(0, [0.5, 0.7])

# Or multiple at once
nurbs_line.insert_knots(0, np.random.rand(10))

#### Knot Removal
This operation is only available for BSplines and NURBS


In [ ]:
bspline_cube.remove_knots(0, [0.5])
print(bspline_cube.knot_vectors)

#### Degree reduction
This operation is only available for BSplines and NURBS


In [ ]:
print("Degrees before : ", bspline_cube.degrees)
bspline_cube.reduce_degrees([1])
print("Degrees after :  ", bspline_cube.degrees)

### Evaluation
Evaluating splines at specific parametric locations

In [ ]:
# Evaluate line
line_points = bezier_line.evaluate([[0.5], [0.6]])

# Or many queries at the same time given a numpy array
bspline_cube_points = bspline_cube.evaluate(
    np.random.rand(10000, 3), nthreads=8
)

### Basis Functions
In order to access basis functions (e.g. for iga-type applications or fitting procedures), use `basis`, `support` or `basis_and_support`:

In [ ]:
queries = np.random.rand(10,1)
basis = nurbs_line.basis(queries)
support = nurbs_line.support(queries)
basis_, support_ = nurbs_line.basis_and_support(queries)
assert np.all(basis == basis_)
assert np.all(support == support_)

You can also use these functions to fill a matrix representing the basis functions at specific positions

In [ ]:
# Create a matrix with basis function values
matrix = np.zeros((queries.shape[0], nurbs_line.control_points.shape[0]))
np.put_along_axis(matrix, support, basis, axis=1)
# Use matrix to compute evaluations
points = matrix @ nurbs_line.control_points
# Here it makes sense to use sparse matrices to save memory and accelerate matrix multiplication
assert np.allclose(points, nurbs_line.evaluate(queries))

### Derivatives
Splinepy provides several functions to compute derivatives of basis functions and fields, both with respect to parametric, but also with respect to physical coordinates. 

Derivatives with respect to physical coordinates are directly available by their respective member functions.

In [ ]:
# Compute first derivative of the field at u=.5
_ = bezier_line.derivative([[0.5]], [1]) 
# Computes second derivative of the field at u=.5
_ = bezier_line.derivative([[0.5]], [2]) 
# Computes first derivative with respect to second parametric axis
_ = bspline_cube.derivative([[0.2, .4, .3]], [0,1,0]) 
# Mixed derivatives
_ = bspline_cube.derivative([[0.2, .4, .3]], [2,1,3]) # Computes B_{,uuvwww}

If required, the jacobian can also be computed directly, returning a matrix for every query.

In [ ]:
# Compute jacobians
jacs = bspline_cube.jacobian(np.random.rand(100,3))
# Compute determinant of jacobians
dets = np.linalg.det(jacs)

The same holds for basis function derivatives

In [ ]:
# Compute seond order derivative of line basis function at position .5
bfd = bezier_line.basis_derivative([[.5]], [2])
# Compute first order debrivative of basis function with respect to v
bfd = bspline_cube.basis_derivative([[0.2, .4, .3]], [0,1,0])

It is also possible to map the derivatives into physical space, using the provided basis function mapper. The mapper supports gradients, divergences, hessian and laplacians (where applicable).

In [ ]:
# Create a scalar field in 3d parametric space (e.g. a temperature field)
field = sp.Bezier(degrees=[2,2,2],control_points=np.random.rand(27,1))
# Set a geometry
mapper = field.mapper(bspline_cube)
# Compute some hessians and laplacians
queries = np.random.rand(1000,3)
results = mapper.basis_function_derivatives(queries, gradient=True, hessian=True, laplacian=True)
# returns a dictionary with entries "support", "gradient", "hessian", "laplacian"

# You can also compute field values in physical space this way
results = mapper.field_derivatives(queries, gradient=True, divergence=False, hessian=True, laplacian=True, basis_function_values=True)
# returns a dictionary with entries "support", "gradient", "hessian", "laplacian", "basis_function_values"

## Proximity search

Proximity search allows to find the closest point within the spline representation, given physical coordinate. 

In [ ]:
closest_points = bspline_cube.proximities(np.random.rand(100,3))

If no suitable approximation can be identified (for example, if some of the points are outside the physical space) a warning is raised, and the best point is returned, if more information is required, use `return_verbose`-flag to trouble shoot. In some cases, it can also help to increase the number of initial samples, that are used as starting points for the local search.

In [ ]:
# Raises warning
closest_points = bspline_cube.proximities(np.random.rand(100,3) * 3)

# Verbose information
verbose_information = bspline_cube.proximities(np.random.rand(100,3) * 3, return_verbose=True)

## Bezier Manipulations
Bezier type spline allow for some special operations for spline modification

### Multiplication
Multiplication between two splines is possible, as long as the dimensions of the physical space are compatible. Multiplication of two splines $A(t)$ and $B(t)$ results in a new spline $C$

$ A(t) * B(t) = C(t) \quad \forall t$

In [ ]:
# Scalar Spline as factor
scalar_spline = sp.Bezier(
    degrees=[2],
    control_points=[[1.],[2],[1]]
)
# Vector Spline as factor
vector_spline = sp.Bezier(
    degrees=[1],
    control_points=[[0.,1.],[3.,0]]
)
# Product
product = vector_spline * scalar_spline


# Check results by evaluating at a random point
eval_query = np.random.rand(10,1)
assert np.allclose(
    product.evaluate(eval_query), 
    (vector_spline.evaluate(eval_query)
     * scalar_spline.evaluate(eval_query))
    )

## Addition
As for the Multiplication, Addition of two splines $A(t)$ and $B(t)$ results in a new spline $C$

$ A(t) + B(t) = C(t) \quad \forall t$

As long as the addition of the spline types is defined.

In [ ]:
# Vector Spline as factor
vector_spline = sp.Bezier(
    degrees=[1],
    control_points=[[0.,1.],[3.,0]]
)

# Create a second spline with different orders
second_spline = vector_spline.copy()
second_spline.elevate_degree(0)

# Sum
sum_spline = vector_spline + second_spline

# Check results by evuating at a random point
eval_query = np.random.rand(10,1)
assert np.allclose(
    sum_spline.evaluate(eval_query), 
    (vector_spline.evaluate(eval_query)
     + vector_spline.evaluate(eval_query))
    )

## Composition
Composition is in the center of microstructure construction. The result of functional composition is a new spline fulfilling 

$ A \circ B = A(B(t))= C(t) \quad \forall t$

Here, the parametric dimension of the outer (or deformation function) must match the physical dimension of the inner function.

In [ ]:
# Inner function (quarter circle)
quarter_circle = sp.RationalBezier(
    degrees=[2],
    control_points=[[1,0],[1,1],[0,1]],
    weights=[1,2**-.5,1]
)

# Outer Function (Rotated Rectangle)
rectangle_surface = sp.Bezier(
    degrees=[1,1],
    control_points=[[.5,0],[1.,.5],[0.,.5],[.5,1.]]
)

# Product
composition = rectangle_surface.compose(quarter_circle)

# Test results
eval_query = np.random.rand(10,1)
assert np.allclose(
    composition.evaluate(eval_query), 
    rectangle_surface.evaluate(
        quarter_circle.evaluate(eval_query)
    )
)

Or into a 3D surface

In [ ]:
# Update Outer Function (Rotated Rectangle)
rectangle_surface = sp.Bezier(
    degrees=[1,1],
    control_points=[[.5,0,0],[1.,.5,.2],[0.,.5,.2],[.5,1.,0]]
)

# Product
composition = rectangle_surface.compose(quarter_circle)

# Test results
eval_query = np.random.rand(10,1)
assert np.allclose(
    composition.evaluate(eval_query), 
    rectangle_surface.evaluate(
        quarter_circle.evaluate(eval_query)
    )
)